# 3. KNN
Result:
- Kaggle score:

Reference:
- http://scikit-learn.org/stable/modules/generated/sklearn.neighbors.KNeighborsClassifier.html#sklearn.neighbors.KNeighborsClassifier
- http://scikit-learn.org/stable/modules/model_evaluation.html#model-evaluation

## Run name

In [1]:
import time
import os
import pandas as pd

project_name = 'Google_LandMark_Rec'
step_name = 'KNN'
time_str = time.strftime("%Y%m%d_%H%M%S", time.localtime())
run_name = project_name + '_' + step_name + '_' + time_str
print('run_name: ' + run_name)

run_name: Google_LandMark_Rec_KNN_20180314_145731


## 项目文件夹

In [2]:
cwd = os.getcwd()
data1_folder = '/data1/kaggle/landmark-recognition-challenge/'
input_folder = os.path.join(data1_folder, 'input')
output_folder = os.path.join(cwd, 'output')
model_folder = os.path.join(cwd, 'model')
feature_folder = os.path.join(cwd, 'feature')
post_pca_feature_folder = os.path.join(cwd, 'post_pca_feature')
log_folder = os.path.join(cwd, 'log')
print('input_folder: \t' + input_folder)
print('output_folder: \t' + output_folder)
print('model_folder: \t' + model_folder)
print('feature_folder: \t' + feature_folder)
print('post_pca_feature_folder: \t' + post_pca_feature_folder)
print('log_folder: \t' + log_folder)

org_train_folder = os.path.join(input_folder, 'org_train')
org_test_folder = os.path.join(input_folder, 'org_test')
train_folder = os.path.join(input_folder, 'data_train')
test_folder = os.path.join(input_folder, 'data_test')
test_sub_folder = os.path.join(test_folder, 'test')

if not os.path.exists(post_pca_feature_folder):
    os.mkdir(post_pca_feature_folder)
    print('Create folder: %s' % post_pca_feature_folder)

input_folder: 	/data1/kaggle/landmark-recognition-challenge/input
output_folder: 	/data2/kaggle/landmark-recognition-challenge/output
model_folder: 	/data2/kaggle/landmark-recognition-challenge/model
feature_folder: 	/data2/kaggle/landmark-recognition-challenge/feature
post_pca_feature_folder: 	/data2/kaggle/landmark-recognition-challenge/post_pca_feature
log_folder: 	/data2/kaggle/landmark-recognition-challenge/log


In [3]:
train_csv_file = os.path.join(input_folder, 'train.csv')
test_csv_file = os.path.join(input_folder, 'test.csv')
sample_submission_folder = os.path.join(input_folder, 'sample_submission.csv')

## 加载feature

In [4]:
%%time
import h5py
import numpy as np
from sklearn.utils import shuffle
np.random.seed(2018)

x_train = []
y_train = {}
x_val = []
y_val = {}
x_test = []


image_size = 150
time_str = '20180311-151108'
cwd = os.getcwd()
# feature_cgg16 = os.path.join(cwd, 'feature', 'feature_VGG16_{}.h5'.format(20180219))
# feature_cgg19 = os.path.join(cwd, 'feature', 'feature_VGG19_{}.h5'.format(20180219))
# feature_resnet50 = os.path.join(cwd, 'feature', 'feature_ResNet50_{}.h5'.format(20180220))
# feature_xception = os.path.join(cwd, 'feature', 'feature_Xception_{}.h5'.format(20180221))
feature_inceptionV3 = os.path.join(cwd, 'post_pca_feature', 'post_pca_feature_InceptionV3_%s_%s.h5' % (image_size, time_str))
# feature_inceptionResNetV2 = os.path.join(cwd, 'feature', 'feature_InceptionResNetV2_%s.h5' % time_str)
# for filename in [feature_cgg16, feature_cgg19, feature_resnet50, feature_xception, feature_inception, feature_inceptionResNetV2]:
for filename in [feature_inceptionV3]:
    with h5py.File(filename, 'r') as h:
        x_train.append(np.array(h['train']))
        y_train = np.array(h['train_labels'])
#         x_val.append(np.array(h['val']))
#         y_val = np.array(h['val_labels'])
        x_test.append(np.array(h['test']))

CPU times: user 232 ms, sys: 372 ms, total: 604 ms
Wall time: 16.4 s


In [5]:
print(x_train[0].shape)
print(len(y_train))
print(x_test[0].shape)

(1219426, 32)
1219426
(115942, 32)


In [6]:
%%time
x_train = np.concatenate(x_train, axis=-1)
x_test = np.concatenate(x_test, axis=-1)
print(x_train.shape)
print(x_test.shape)

(1219426, 32)
(115942, 32)
CPU times: user 96 ms, sys: 128 ms, total: 224 ms
Wall time: 221 ms


In [7]:
from sklearn.model_selection import train_test_split
x_train, x_val, y_train, y_val = train_test_split(x_train, y_train, test_size=0.0025, random_state=5)

print(x_train.shape)
print(x_val.shape)
print(y_train.shape)
print(y_val.shape)

(1216377, 32)
(3049, 32)
(1216377,)
(3049,)


## Train KNN with GridSearchCV

In [8]:
set_y_train = list(set(y_train))
print(len(set_y_train))

14951


In [15]:
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score

def my_GridSearchCV(model, parameters):
    if parameters:
        clf = GridSearchCV(model, parameters)
    else:
        clf = model
    clf.fit(x_train, y_train)
    return clf

def my_train(models):
    clfs = {}
    for key, value in models.items():
        print('start ' + key)
        t0 = time.time()
        clf = my_GridSearchCV(value['model'], value['param'])
        t1 = time.time()
        print('%s spend time: %.2f s' %(key, t1-t0))
        print(clf.best_estimator_)
        clfs[key] = clf
    return clfs

def my_predict(clfs, x_val=x_val, y_val=y_val):
    preds = {}
    for key, value in clfs.items():
        print(key, end=':  ')
        y_pred = value.predict(x_val)
        y_train_pred = value.predict(x_train[:10000])
        train_acc = accuracy_score(y_train[:10000], y_train_pred)
        print('train_acc: %.2f' % train_acc)
        preds[key] = accuracy_score(y_val, y_pred)
        print('val_acc: %.2f' % preds[key])
    return preds

In [ ]:
%%time
from sklearn.svm import SVC
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier

models = {
#     'SVC': {
#         'model': SVC(),
#         'param': {'kernel':('linear', 'rbf'), 'C':[1, 5, 10]}
#     },
#     'GaussianNB':{
#         'model': GaussianNB(),
#         'param': {}
#     },
    'RandomForestClassifier':{
        'model': RandomForestClassifier(min_samples_leaf=30, n_jobs=-1),
        'param': {'n_estimators': [10, 20, 50], 'max_depth':[10, 20]}
    },
    'KNeighborsClassifier':{
        'model': KNeighborsClassifier(weights='distance', algorithm='ball_tree', leaf_size=30, p=2, metric='minkowski', n_jobs=-1),
        'param': {'n_neighbors': [10, 20, 50]}
    }
}

clfs = my_train(models)
preds = my_predict(clfs)

start RandomForestClassifier


/root/anaconda3/lib/python3.6/site-packages/sklearn/model_selection/_split.py:605: Warning: The least populated class in y has only 1 members, which is too few. The minimum number of members in any class cannot be less than n_splits=3.
  % (min_groups, self.n_splits)), Warning)


In [ ]:
%%time
import time
import os

selected_clf_name = 'RandomForestClassifier'
clf = clfs[selected_clf_name]
run_name_acc = run_name + '_' + str(int(preds[selected_clf_name]*10000)).zfill(4)
print(run_name_acc)

pred_file = os.path.join(output_folder, run_name_acc + '.csv')
print(pred_file)

## Predict

In [51]:
%%time
y_test = clf.predict(x_test)
print(y_test.shape)

(115942,)
CPU times: user 18min 58s, sys: 80 ms, total: 18min 58s
Wall time: 4min 44s


In [59]:
print(clf.best_estimator_)

KNeighborsClassifier(algorithm='ball_tree', leaf_size=30, metric='minkowski',
           metric_params=None, n_jobs=-1, n_neighbors=10, p=2,
           weights='distance')


In [58]:
# a = list(set(y_test))
print(len(a))
print(a[:10])

3902
[3, 5, 6, 12, 16, 18, 19, 22, 26, 30]


In [54]:
%%time
y_test = clf.predict_proba(x_test)
print(y_test.shape)

(115942, 5025)
CPU times: user 13min 9s, sys: 14.7 s, total: 13min 24s
Wall time: 3min 21s


In [55]:
print(y_test[0])

[0. 0. 0. ... 0. 0. 0.]


In [48]:
sample_submission_csv = pd.read_csv(sample_submission_folder)
print('sample_submission_csv.shape is {0}.'.format(sample_submission_csv.shape))
display(sample_submission_csv.head(2))

sample_submission_csv.shape is (117703, 2).


,id,landmarks
0,000088da12d664db,8815 0.03
1,0001623c6d808702,7249 0.61


In [49]:
# 这里证明os.listdir()得到的图片名称list不正确
files = os.listdir(os.path.join(input_folder, 'data_test', 'test'))
print(files[:10])

['9b77bc42cb82a1b0.jpg', '5c877399eccc61a4.jpg', '37bd287eaacb135d.jpg', '11be3fa179d99444.jpg', 'a9fc808636a33c00.jpg', 'e56552a892c04319.jpg', '4f392ea999647b8d.jpg', '4c72be08bd8f1131.jpg', '1fac42c252050895.jpg', 'ec8d5b5fab741921.jpg']


In [50]:
# 这里证明ImageDataGenerator()得到的图片名称list才是正确
from keras.preprocessing.image import ImageDataGenerator
gen = ImageDataGenerator()
image_size = (299, 299)
batch_size = 128
test_generator  = gen.flow_from_directory(test_folder, image_size, shuffle=False, batch_size=batch_size)
print('test_generator')
print(len(test_generator.filenames))
print(test_generator.filenames[:10])

Found 115942 images belonging to 1 classes.
test_generator
115942
['test/000088da12d664db.jpg', 'test/0001623c6d808702.jpg', 'test/0001bbb682d45002.jpg', 'test/0002362830cfe3a3.jpg', 'test/000270c9100de789.jpg', 'test/0002b0fab5d3ccc4.jpg', 'test/000396be3c24830a.jpg', 'test/000506dc6ab3a40e.jpg', 'test/0005292fc4b005a3.jpg', 'test/0005456a82264bc8.jpg']


In [46]:
%%time
max_indexes = np.argmax(y_test, -1)
print(max_indexes.shape)

test_dict = {}
for i, paire in enumerate(zip(test_generator.filenames, max_indexes)):
    image_name, label = paire[0], paire[1]
    image_id = image_name[5:-4]
    test_dict[image_id] = '%d %.4f' % (indx, y_pred[i, indx])
    test_dict[image_id] = '%d %.4f' % (label, 1)

#确认图片的id是否能与ImageDataGenerator()对应上
for key in list(test_dict.keys())[:10]:
    print('%s  %s' % (key, test_dict[key]))

000088da12d664db  11063 1.0000
0001623c6d808702  13005 1.0000
0001bbb682d45002  6051 1.0000
0002362830cfe3a3  5554 1.0000
000270c9100de789  4404 1.0000
0002b0fab5d3ccc4  9633 1.0000
000396be3c24830a  1581 1.0000
000506dc6ab3a40e  5379 1.0000
0005292fc4b005a3  10409 1.0000
0005456a82264bc8  9587 1.0000
CPU times: user 2.7 s, sys: 24 ms, total: 2.72 s
Wall time: 681 ms


In [ ]:
%%time
len_sample_submission_csv = len(sample_submission_csv)
print('len(len_sample_submission_csv)=%d' % len_sample_submission_csv)
count = 0
for i in range(len_sample_submission_csv):
    image_id = sample_submission_csv.iloc[i, 0]
#     landmarks = sample_submission_csv.iloc[i, 1]
    if image_id in test_dict:
        pred_landmarks = test_dict[image_id]
#         print('%s  %s' % (image_id, pred_landmarks))
        sample_submission_csv.iloc[i, 1] = pred_landmarks
    else:
#         print(image_id)
#         sample_submission_csv.iloc[i, 1] = '9633 1.0' # 属于9633的类最多，所以全都设置成这个类，可能会比设置成空得到的结果好
        sample_submission_csv.iloc[i, 1] = '' # 设置成空
    count += 1
    if count % 10000 == 0:
        print(int(count/10000), end=' ')
display(sample_submission_csv.head(2))

In [ ]:
pred_file = os.path.join(output_folder, 'pred_' + run_name_acc + '.csv')
sample_submission_csv.to_csv(pred_file, index=None)

In [ ]:
print(run_name_acc)
print('Done !')